#### 숫자예측 모델(LSTM)

In [8]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

AttributeError: module 'pandas' has no attribute 'Series'

In [ ]:
# Data (random num sequence)
data = np.array([i for i in range(1,21)])
timesteps = 3 # 타임스텝 길이

In [ ]:
x_data = []
y_data = []

In [2]:
# 타임스텝에 따라 입력 데이터와 출력 데이터 생성
for i in range(len(data) - timesteps):
    x_data.append(data[i:i+timesteps]) #입력(2개의 숫자)
    y_data.append(data[i + timesteps]) # 출력(다음숫자)
    
x_data = np.array(x_data).reshape(-1, timesteps, 1) # (sample num, timestep, feature num)
y_data = np.array(x_data) # 출력값

NameError: name 'data' is not defined

In [4]:
# LSTM model definition
model = Sequential([
    LSTM(10, input_shape=(timesteps, 1)), # lstm layer(unit : 10)
    Dense(1) # 출력 layer
])

NameError: name 'Sequential' is not defined

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
# 3. 모델 학습
model.fit(x_data, y_data, epochs=1000, verbose=1)  # 100번 반복 학습

In [ ]:
# 4. 예측 테스트
test_input = np.array([18, 19, 20]).reshape(1, timesteps, 1)  # 입력 예시
predicted_value = model.predict(test_input)

In [ ]:
# 결과 출력
print("입력 데이터:", test_input.flatten()) # flatten함수 : [[1,2],[3,4]] --> [1,2,3,4]
print("예측 값:", predicted_value[0][0])

In [1]:
# sys.path 확인
# Jupyter Notebook에서 현재 Python 경로에 포함된 패키지 경로를 확인하세요.
import sys
print(sys.executable)

c:\AI_project\.venv\Scripts\python.exe


In [2]:
# TensorFlow 직접 로드 확인
# Jupyter Notebook에서 TensorFlow를 직접 불러와 문제를 확인하세요.
import tensorflow as tf
print(tf.__version__)

2.18.0


In [12]:
import pandas as pd
print(pd.__version__)

AttributeError: module 'pandas' has no attribute '__version__'

In [11]:
import sys
print(sys.path)

['C:\\Python3.106\\python310.zip', 'C:\\Python3.106\\DLLs', 'C:\\Python3.106\\lib', 'C:\\Python3.106', 'c:\\AI_project\\.venv', '', 'c:\\AI_project\\.venv\\lib\\site-packages', 'c:\\AI_project\\.venv\\lib\\site-packages\\win32', 'c:\\AI_project\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\AI_project\\.venv\\lib\\site-packages\\Pythonwin']
